In [ ]:
from IPython.display import display, HTML
import os
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
%run nb_core/root_dirs.ipynb
setup_syspath_disentangle(False)
%run nb_core/disentangle_imports.ipynb

In [ ]:
ckpt_dir = "/group/jug/ashesh/training/disentangle/2503/D17-M3-S0-L0/5"
assert os.path.exists(ckpt_dir)

In [ ]:
def get_dtype(ckpt_fpath):
    if os.path.isdir(ckpt_fpath):
        ckpt_fpath = ckpt_fpath[:-1] if ckpt_fpath[-1] == '/' else ckpt_fpath
    elif os.path.isfile(ckpt_fpath):
        ckpt_fpath = os.path.dirname(ckpt_fpath)
    assert ckpt_fpath[-1] != '/'
    return int(ckpt_fpath.split('/')[-2].split('-')[0][1:])

In [ ]:
dtype = get_dtype(ckpt_dir)
dtype

In [ ]:
from disentangle.data_loader.patch_index_manager import TilingMode

image_size_for_grid_centers = 64
mmse_count = 1
custom_image_size = None
use_selected_fpaths = ['Test1_Slice2_a/4.nd2']
data_t_list = None #[0,1,2,3]
tiling_mode = TilingMode.ShiftBoundary

save_comparative_plots =False
enable_calibration = False
batch_size = 16
num_workers = 4
COMPUTE_LOSS = False
use_deterministic_grid = None
threshold = None # 0.02
compute_kl_loss = False
evaluate_train = False# inspect training performance
eval_datasplit_type = DataSplitType.All 
val_repeat_factor = None
psnr_type = 'range_invariant' #'simple', 'range_invariant'

In [ ]:
%run nb_core/config_loader.ipynb

In [ ]:
tokens = ckpt_dir.split('/')
idx = tokens.index('disentangle')
if config.model.model_type == 25 and tokens[idx+1] == '2312':
    config.model.model_type = ModelType.LadderVAERestrictedReconstruction

In [ ]:
from disentangle.core.sampler_type import SamplerType
from disentangle.core.loss_type import LossType
from disentangle.data_loader.ht_iba1_ki67_rawdata_loader import SubDsetType
# from disentangle.core.lowres_merge_type import LowresMergeType


with config.unlocked():
    if 'depth3D' in config.data and config.data.depth3D > 1:
        config.data.mode_3D = True
        config.model.mode_3D = True
        
    config.model.skip_nboundary_pixels_from_loss = None
    if config.model.model_type == ModelType.UNet and 'n_levels' not in config.model:
        config.model.n_levels = 4
    if config.data.sampler_type == SamplerType.NeighborSampler:
        config.data.sampler_type = SamplerType.DefaultSampler
        config.loss.loss_type = LossType.Elbo
        config.data.grid_size = config.data.image_size
    if 'ch1_fpath_list' in config.data:
        config.data.ch1_fpath_list = config.data.ch1_fpath_list[:1]
        config.data.mix_fpath_list = config.data.mix_fpath_list[:1]
    if config.data.data_type == DataType.Pavia2VanillaSplitting:
        if 'channel_2_downscale_factor' not in config.data:
            config.data.channel_2_downscale_factor = 1
    if config.model.model_type == ModelType.UNet and 'init_channel_count' not in config.model:
        config.model.init_channel_count = 64
    
    if 'skip_receptive_field_loss_tokens' not in config.loss:
        config.loss.skip_receptive_field_loss_tokens = []
    
    if dtype == DataType.HTIba1Ki67:
        config.data.subdset_type = SubDsetType.OnlyIba1P30
        config.data.empty_patch_replacement_enabled = False
    
    if 'lowres_merge_type' not in config.model.encoder:
        config.model.encoder.lowres_merge_type = 0
    if 'validtarget_random_fraction' in config.data:
        config.data.validtarget_random_fraction = None
    
    if config.data.data_type == DataType.TwoDset:
        config.model.model_type = ModelType.LadderVae
        for key in config.data.dset1:
            config.data[key] = config.data.dset1[key]
    
    if 'dump_kth_frame_prediction' in config.training:
        config.training.dump_kth_frame_prediction = None

    if 'input_is_sum' not in config.data:
        config.data.input_is_sum = False

## Replace the dataset

In [ ]:
from disentangle.data_loader.sox2golgi_v2_rawdata_loader import Sox2GolgiV2ChannelList
from disentangle.data_loader.multifile_raw_dloader import SubDsetType
import ml_collections
with config.unlocked():
    config.data.data_type = DataType.TavernaSox2GolgiV2
    config.data.subdset_type = SubDsetType.MultiChannel
    # all channels: ['555-647', 'GT_Cy5', 'GT_TRITC']
    config.data.channel_idx_list = [
            Sox2GolgiV2ChannelList.GT_TRITC,Sox2GolgiV2ChannelList.GT_Cy5, Sox2GolgiV2ChannelList.GT_555_647
        ]
    config.data.num_channels = len(config.data.channel_idx_list)
    config.data.input_idx = 2
    if use_selected_fpaths is not None:
        config.data.use_selected_fpaths = use_selected_fpaths
    config.data.target_idx_list = [0, 1]
    config.model.num_targets = len(config.data.target_idx_list)

In [ ]:
dtype = config.data.data_type

if dtype in [DataType.CustomSinosoid, DataType.CustomSinosoidThreeCurve]:
    data_dir = f'{DATA_ROOT}/sinosoid_without_test/sinosoid/'
elif dtype == DataType.OptiMEM100_014:
    data_dir = f'{DATA_ROOT}/microscopy/'
elif dtype == DataType.Prevedel_EMBL:
    data_dir = f'{DATA_ROOT}/Prevedel_EMBL/PKG_3P_dualcolor_stacks/NoAverage_NoRegistration/'
elif dtype == DataType.AllenCellMito:
    data_dir = f'{DATA_ROOT}/allencell/2017_03_08_Struct_First_Pass_Seg/AICS-11/'
elif dtype == DataType.SeparateTiffData:
    data_dir = f'{DATA_ROOT}/ventura_gigascience'
elif dtype == DataType.SemiSupBloodVesselsEMBL:
    data_dir = f'{DATA_ROOT}/EMBL_halfsupervised/Demixing_3P'
elif dtype == DataType.Pavia2VanillaSplitting:
    data_dir = f'{DATA_ROOT}/pavia2'
# elif dtype == DataType.ExpansionMicroscopyMitoTub:
    # data_dir = f'{DATA_ROOT}/expansion_microscopy_Nick/'
elif dtype == DataType.ShroffMitoEr:
    data_dir = f'{DATA_ROOT}/shrofflab/'
elif dtype == DataType.HTIba1Ki67:
    data_dir = f'{DATA_ROOT}/Stefania/20230327_Ki67_and_Iba1_trainingdata/'
elif dtype == DataType.BioSR_MRC:
    data_dir = f'{DATA_ROOT}/BioSR/'
elif dtype == DataType.ExpMicroscopyV2:
    data_dir = f'{DATA_ROOT}/expansion_microscopy_v2/datafiles/'
elif dtype == DataType.TavernaSox2Golgi:
    data_dir = f'{DATA_ROOT}/TavernaSox2Golgi/'
elif dtype == DataType.TavernaSox2GolgiV2:
    data_dir = f'{DATA_ROOT}/TavernaSox2Golgi/acquisition2/'
elif dtype == DataType.Pavia3SeqData:
    data_dir = f'{DATA_ROOT}/pavia3_sequential/'
elif dtype == DataType.NicolaData:
    data_dir = f'{DATA_ROOT}/nikola_data/20240531/'
elif dtype == DataType.Dao3ChannelWithInput:
    data_dir = f'{DATA_ROOT}/Dao4Channel/'
elif dtype == DataType.Dao3Channel:
    data_dir = f'{DATA_ROOT}/Dao3Channel/'
elif dtype == DataType.SilvioLabCSHLData:
    data_dir = f'{DATA_ROOT}/svilen_cshl2024/'
elif dtype == DataType.ExpMicroscopyV3:
    data_dir = f'{DATA_ROOT}/expansion_microscopy_v4/405_NHS_488BODIPY/'
elif dtype == DataType.Elisa3DData:
    data_dir = f'{DATA_ROOT}/Elisa3D/'

In [ ]:
# config.model.noise_model_ch1_fpath = config.model.noise_model_ch1_fpath.replace('/home/ashesh.ashesh/training/', '/group/jug/ashesh/training_pre_eccv/')
# config.model.noise_model_ch2_fpath = config.model.noise_model_ch2_fpath.replace('/home/ashesh.ashesh/training/', '/group/jug/ashesh/training_pre_eccv/')

In [ ]:
%run nb_core/disentangle_setup.ipynb

In [ ]:
if config.data.multiscale_lowres_count is not None and custom_image_size is not None:
    model.reset_for_different_output_size(custom_image_size)

In [ ]:
idx = np.random.randint(len(val_dset))
inp_tmp, tar_tmp, *_ = val_dset[idx]
ncols = len(tar_tmp)
nrows = 2
_,ax = plt.subplots(figsize=(4*ncols,4*nrows),ncols=ncols,nrows=nrows)
for i in range(min(ncols,len(inp_tmp))):
    ax[0,i].imshow(inp_tmp[i])

for channel_id in range(ncols):
    ax[1,channel_id].imshow(tar_tmp[channel_id])

In [ ]:
if data_t_list is not None:
    val_dset.reduce_data(t_list=data_t_list)

In [ ]:
# # high val dset 
# import ml_collections
# new_config = ml_collections.ConfigDict(config)
# highsnr_val_dset = None
# if 'poisson_noise_factor' in new_config.data and new_config.data.poisson_noise_factor > 0:
#     new_config.data.poisson_noise_factor = -1
#     _, highsnr_val_dset = create_dataset(new_config, data_dir, eval_datasplit_type=eval_datasplit_type,
#                                         kwargs_dict=dloader_kwargs)



In [ ]:
def get_full_input_frame(idx, dset):
    img_tuples, noise_tuples = dset._load_img(idx)
    if len(noise_tuples) > 0:
        factor = np.sqrt(2) if dset._input_is_sum else 1.0
        img_tuples = [x + noise_tuples[0] * factor for x in img_tuples]

    inp = 0
    for nch in img_tuples:
        inp += nch/len(img_tuples)
    h_start, w_start = dset._get_deterministic_hw(idx)
    return inp, h_start, w_start


In [ ]:
from disentangle.core.tiff_reader import load_tiff
from disentangle.analysis.paper_plots import show_for_one, get_plotoutput_dir
def get_hwt_start(idx):
    h,w,t = val_dset.idx_manager.hwt_from_idx(idx, grid_size=64)
    print(h,w,t)
    pad = val_dset.per_side_overlap_pixelcount()
    h =  h - pad
    w = w - pad
    return h,w,t

def get_crop_from_fulldset_prediction(full_dset_pred, idx, patch_size=256):
    h,w,t = get_hwt_start(idx)
    return np.swapaxes(full_dset_pred[t,h:h+patch_size,w:w+patch_size].astype(np.float32)[None], 0, 3)[...,0]

if save_comparative_plots:
    assert eval_datasplit_type == DataSplitType.Test
    # CCP vs Microtubules: 925, 659, 502
    # hdn_usplitdata = load_tiff('/group/jug/ashesh/data/paper_stats/Test_PNone_G16_M3_Sk0/pred_disentangle_2402_D23-M3-S0-L0_67.tif')
    hdn_usplitdata = load_tiff('/group/jug/ashesh/data/paper_stats/Test_PNone_G32_M5_Sk0/pred_disentangle_2403_D23-M3-S0-L0_29.tif')

    # ER vs Microtubule 853, 859, 332
    # hdn_usplitdata = load_tiff('/group/jug/ashesh/data/paper_stats/Test_PNone_G16_M3_Sk0/pred_disentangle_2402_D23-M3-S0-L0_60.tif')

    #  ER vs CCP 327, 479, 637, 568
    # hdn_usplitdata = load_tiff('/group/jug/ashesh/data/paper_stats/Test_PNone_G16_M3_Sk0/pred_disentangle_2402_D23-M3-S0-L0_59.tif')

    #  F-actin vs ER 797
    # hdn_usplitdata = load_tiff('/group/jug/ashesh/data/paper_stats/Test_PNone_G32_M10_Sk0/pred_disentangle_2403_D23-M3-S0-L0_15.tif')

    idx = 10#np.random.randint(len(val_dset))
    patch_size = 500
    mmse_count = 50
    print(idx)
    show_for_one(idx, val_dset, highsnr_val_dset, model, None, mmse_count=mmse_count, patch_size=patch_size, baseline_preds=[
        get_crop_from_fulldset_prediction(hdn_usplitdata, idx).astype(np.float32),
    ], num_samples=0)


    plotsdir = get_plotoutput_dir(ckpt_dir, patch_size, mmse_count=mmse_count)
    model_id = ckpt_dir.strip('/').split('/')[-1]
    fname = f'patch_comparison_{idx}_{model_id}.png'
    fpath = os.path.join(plotsdir, fname)
    plt.savefig(fpath, dpi=200, bbox_inches='tight')
    print(f'Saved to {fpath}')

## Self-supervision
Here, we now do some self-supervised finetuning. 

In [ ]:
import torch

def zero_mean(x):
    return x - torch.mean(x, dim=1, keepdim=True)


def fix_range(gt, x):
    a = torch.sum(gt * x, dim=1, keepdim=True) / (torch.sum(x * x, dim=1, keepdim=True))
    return x * a


def fix(gt, x):
    return fix_range(gt, zero_mean(x))

def RangeInvariantPsnr(gt, pred):
    """
    NOTE: Works only for grayscale images.
    Adapted from https://github.com/juglab/ScaleInvPSNR/blob/master/psnr.py
    It rescales the prediction to ensure that the prediction has the same range as the ground truth.
    """
    assert len(gt.shape) == 3, 'Images must be in shape: (batch,H,W)'
    gt = gt.view(len(gt), -1)
    pred = pred.view(len(gt), -1)
    # ra = (torch.max(gt, dim=1).values - torch.min(gt, dim=1).values) / torch.std(gt, dim=1)
    gt_ = zero_mean(gt) / torch.std(gt, dim=1, keepdim=True)
    return _PSNR_internal(zero_mean(gt_), fix(gt_, pred), ra)


In [ ]:
from tqdm import tqdm
std_arr = []
for i in tqdm(range(len(val_dset))):
    inp_1, tar_1  = val_dset[i]
    std_arr.append(np.std(inp_1))

In [ ]:
from finetunesplit.asymmetric_transforms import TransformAllChannels, VFlip, Rotate, HFlip
transform_types = {0:[VFlip(), Rotate(),HFlip()], 1:[ VFlip(), HFlip(), Rotate()]}
transform_all = TransformAllChannels(transform_types)
content_std = np.quantile(std_arr, 0.99)
idx_list = (np.where(np.array(std_arr) > content_std)[0]).tolist()

In [ ]:
inp, tar = val_dset[idx_list[0]]

In [ ]:
_,ax = plt.subplots(figsize=(9,3),ncols=3)
ax[0].imshow(inp[0])
ax[1].imshow(tar[0])
ax[2].imshow(tar[1])

In [ ]:
from finetunesplit.asymmetric_transforms import get_inverse_transforms
with torch.no_grad():
    pred_one, _ = model(torch.Tensor(inp[None]).cuda())
    transformed_pred, applied_transforms = transform_all(pred_one[:,:2])

    t_val = 0.7
    transformed_inp = transformed_pred[:,:1]* t_val + transformed_pred[:,1:] * (1-t_val)
    # transformed_inp = (transformed_inp - transformed_inp.mean().item())/transformed_inp.std().item()
    # transformed_inp = transformed_inp*inp.std() + inp.mean()
    transformed_inp_pred, _ = model(transformed_inp)
    inv_transform = get_inverse_transforms(applied_transforms)
    inv_transformed_pred, _  = transform_all(transformed_inp_pred, params_dict = inv_transform, inverse=True)

    # 
    inv_inp = inv_transformed_pred[:,:1]* t_val + inv_transformed_pred[:,1:] * (1-t_val)

In [ ]:
print('Orig', inp.mean(), inp.std())
print('Transformed', transformed_inp.mean().item(), transformed_inp.std().item())

In [ ]:
_,ax = plt.subplots(figsize=(21,6),ncols=7,nrows=2)

ax[0,0].imshow(inp[0])
ax[0,0].set_title('x')

# original prediction
ax[0,1].set_title('Pred(x)')
ax[0,1].imshow(pred_one[0,0].detach().cpu().numpy())
ax[1,1].imshow(pred_one[0,1].detach().cpu().numpy())

# transformed prediction
ax[0,2].set_title('T(Pred(x))')
ax[0,2].imshow(transformed_pred[0,0].detach().cpu().numpy())
ax[1,2].imshow(transformed_pred[0,1].detach().cpu().numpy())

ax[0,3].set_title('x2 = Inp(T(Pred(x)))')
ax[0,3].imshow(transformed_inp[0,0].detach().cpu().numpy())

ax[0,4].set_title('$T^{-1}$(Pred(x2))')
ax[0,4].imshow(inv_transformed_pred[0,0].detach().cpu().numpy())
ax[1,4].imshow(inv_transformed_pred[0,1].detach().cpu().numpy())

ax[0,5].set_title('Target')
ax[0,5].imshow(tar[0])
ax[1,5].imshow(tar[1])

ax[0,5].set_ylabel('Channel 1')
ax[1,5].set_ylabel('Channel 2')
# make the label right side
ax[0,5].yaxis.set_label_position("right")
ax[1,5].yaxis.set_label_position("right")
ax[1,0].axis('off')
ax[1,3].axis('off')

ax[0,6].set_title('$x_{rec}$ = Inp($T^{-1}$(Pred(x2)))')
ax[0,6].imshow(inv_inp[0,0].detach().cpu().numpy())
ax[1,6].axis('off')
plt.tight_layout()

In [ ]:
from disentangle.analysis.stitch_prediction import stitch_predictions
from disentangle.analysis.mmse_prediction import get_dset_predictions
# from disentangle.analysis.stitch_prediction import get_predictions as get_dset_predictions

pred_tiled, rec_loss, logvar_tiled, patch_psnr_tuple, pred_std_tiled = get_dset_predictions(model, val_dset,batch_size,
                                               num_workers=num_workers,
                                               mmse_count=mmse_count,
                                                model_type = config.model.model_type,
                                              )
tmp = np.round([x.item() for x in patch_psnr_tuple],2)
print('Patch wise PSNR, as computed during training', tmp,np.mean(tmp))

In [ ]:
pred_tiled.shape

In [ ]:
idx_list = np.where(logvar_tiled.squeeze() < -6)[0]
if len(idx_list) > 0:
    plt.imshow(val_dset[idx_list[0]][1][1])

In [ ]:
len(val_dset)

In [ ]:
print(np.quantile(rec_loss, [0,0.01,0.5, 0.9,0.99,0.999,1]).round(2))

In [ ]:
if pred_tiled.shape[-1] != val_dset.get_img_sz():
    pad = (val_dset.get_img_sz() - pred_tiled.shape[-1] )//2
    pred_tiled = np.pad(pred_tiled, ((0,0),(0,0),(pad,pad),(pad,pad)))

pred = stitch_predictions(pred_tiled,val_dset, )
if len(np.unique(logvar_tiled)) == 1:
    logvar = None
else:
    logvar = stitch_predictions(logvar_tiled,val_dset, )
pred_std = stitch_predictions(pred_std_tiled,val_dset, )

In [ ]:
pred[0].shape

In [ ]:
is_list_prediction = isinstance(pred, list)
tar_unnorm = (val_dset._data if not is_list_prediction else [val_dset.dsets[i]._data for i in range(len(val_dset.dsets))])

In [ ]:
tar_unnorm[0].shape

In [ ]:
# if 'target_idx_list' in config.data and config.data.target_idx_list is not None:
#     pred = pred[...,:len(config.data.target_idx_list)]
#     pred_std = pred_std[...,:len(config.data.target_idx_list)]
nTar = pred[0].shape[-1]

if "target_idx_list" in config.data and config.data.target_idx_list is not None:
    nTar =len(config.data.target_idx_list)
    pred = pred[..., :len(config.data.target_idx_list)] if not is_list_prediction else [pred[i][..., :len(config.data.target_idx_list)] for i in range(len(pred))]
    pred_std = pred_std[...,:len(config.data.target_idx_list)] if not is_list_prediction else [pred_std[i][..., :len(config.data.target_idx_list)] for i in range(len(pred_std))]

if 'input_idx' in config.data and config.data.input_idx is not None:
    inp_unnorm = [x[...,config.data.input_idx] for x in tar_unnorm]
    tar_unnorm = [x[...,:nTar] for x in tar_unnorm]



### Optimal Estimation of t

In [ ]:
from disentangle.core.psnr import RangeInvariantPsnr

In [ ]:
import numpy as np
from tqdm import tqdm
mean_psnr_arr = []
std_psnr_arr = []
t_values = np.arange(0.0,1.0, 0.05) 
for t in tqdm(t_values):
    inp_tiled = [(t *pred[i][...,0] + (1-t)*pred[i][...,1]) for i in range(len(pred))]
    psnr_values = [RangeInvariantPsnr(inp_unnorm[i]*1.0, inp_tiled[i]).item() for i in range(len(inp_unnorm))]
    mean_psnr_arr.append(np.mean(psnr_values))
    std_psnr_arr.append(np.std(psnr_values))

In [ ]:
best_idx = np.argmax(mean_psnr_arr)
print(f'Best t value: {t_values[best_idx]}')

In [ ]:
plt.plot(t_values, mean_psnr_arr)

In [ ]:
_,ax  = plt.subplots(figsize=(8,4),ncols=2)
ax[0].imshow(inp_tiled[0][0])
ax[1].imshow(inp_unnorm[0][0])

In [ ]:
def print_ignored_pixels():
    ignored_pixels = 1
    while(pred[0,-ignored_pixels:,-ignored_pixels:,].std() ==0):
        ignored_pixels+=1
    ignored_pixels-=1
    print(f'In {pred.shape}, last {ignored_pixels} many rows and columns are all zero.')
    return ignored_pixels

# actual_ignored_pixels = print_ignored_pixels()

In [ ]:
sep_mean, sep_std = model.data_mean["target"], model.data_std["target"]
sep_mean = sep_mean.squeeze().reshape(1, 1, 1, -1)
sep_std = sep_std.squeeze().reshape(1, 1, 1, -1)

In [ ]:
pred[0].shape, sep_mean.shape, sep_std.shape

In [ ]:
if is_list_prediction:
    pred_unnorm = [pred[i] * sep_std.cpu().numpy() + sep_mean.cpu().numpy() for i in range(len(pred))]
else:
    pred_unnorm = pred * sep_std.cpu().numpy() + sep_mean.cpu().numpy()

if ckpt_dir == '/group/jug/ashesh/training/disentangle/2404/D17-M3-S0-L8/4':
    print("Reversing the order of channels")
    pred_unnorm = [x[...,::-1] for x in pred_unnorm]


In [ ]:
from disentangle.scripts.evaluate import compute_high_snr_stats
stats_dict = compute_high_snr_stats(tar_unnorm, pred_unnorm)

In [ ]:
import matplotlib.patches as patches
ncols = pred_unnorm[0].shape[-1]
imgsz = 4
_,ax = plt.subplots(figsize=((1+ncols)*imgsz,2*imgsz),nrows=2,ncols=ncols + 1)
img_idx = np.random.randint(len(tar_unnorm))
sz = 600

hs = np.random.randint(tar_unnorm[0].shape[1]-sz)
ws = np.random.randint(tar_unnorm[0].shape[2]-sz)
print(img_idx, hs, ws)

for i in range(ncols):
    vmin = tar_unnorm[img_idx][0,hs:hs+sz, ws:ws+sz ,i].min()
    vmax = tar_unnorm[img_idx][0,hs:hs+sz, ws:ws+sz ,i].max()
    ax[0,i+1].imshow(tar_unnorm[img_idx][0,hs:hs+sz, ws:ws+sz ,i], vmin=vmin, vmax=vmax)
    ax[1,i+1].imshow(pred_unnorm[img_idx][0,hs:hs+sz, ws:ws+sz,i], vmin=vmin, vmax=vmax)

if 'input_idx' in config.data and config.data.input_idx is not None:
    inp = inp_unnorm[img_idx][0]
else:
    inp = np.mean(tar_unnorm[img_idx][0], axis=-1)

    
ax[0,0].imshow(inp)
rect = patches.Rectangle((ws, hs), sz,sz, linewidth=2, edgecolor='r', facecolor='none')
ax[0,0].add_patch(rect)

ax[1,0].imshow(inp[hs:hs+sz, ws:ws+sz])
plt.subplots_adjust(wspace=0.03, hspace=0.03)
ax[0,0].set_title('Input')
twinx = ax[0,-1].twinx()
twinx.set_ylabel('Target')
clean_ax(twinx)
twinx = ax[1,-1].twinx()
clean_ax(twinx)
twinx.set_ylabel('Prediction')
clean_ax(ax)


In [ ]:
break here

## SSL finetuning. 

In [ ]:
from torch.utils.data import DataLoader
from finetunesplit.loss import SSL_loss
def pred_func(inp):
    return model(inp)[0][:,:2]


# define a learnable scalar and an offset 
scalar1 = torch.nn.Parameter(torch.tensor(1.0).cuda())
offset1 = torch.nn.Parameter(torch.tensor(0.0).cuda())

scalar2 = torch.nn.Parameter(torch.tensor(1.0).cuda())
offset2 = torch.nn.Parameter(torch.tensor(0.0).cuda())
mixing_ratio = torch.nn.Parameter(torch.tensor(0.7).cuda())
# define an optimizer
# opt = torch.optim.Adam(model.parameters(), lr=1e-3)
opt = torch.optim.Adamax([scalar1, offset1, scalar2, offset2, mixing_ratio], lr=1e-3, weight_decay=0)

max_step_count = 50000
batch_size = 16

loss_arr = []
best_loss = 1e6
best_scalars = None
best_offsets = None

scalar1_arr = []
offset1_arr = []

scalar2_arr = []
offset2_arr = []
mixing_ratio_arr= []
cnt = 0
while True:
    dloader = DataLoader(val_dset, batch_size=batch_size, shuffle=True, num_workers=4)
    for i, (inp, tar) in tqdm(enumerate(dloader)):
        if i >= max_step_count:
            break
        inp = inp.cuda()
        # reset the gradients
        opt.zero_grad()

        loss_dict = SSL_loss(pred_func, inp, transform_all, mixing_ratio=mixing_ratio, scalar1=scalar1, offset1=offset1, scalar2=scalar2, offset2=offset2)
        loss = loss_dict['loss_inp']
        loss.backward()
        loss_arr.append(loss.item())
        
        scalar1_arr.append(scalar1.item())
        offset1_arr.append(offset1.item())
        scalar2_arr.append(scalar2.item())
        offset2_arr.append(offset2.item())
        cnt += len(inp)
        mixing_ratio_arr.append(mixing_ratio.item())
        opt.step()
        if loss.item() < best_loss:
            best_loss = loss.item()
            print(f'Loss: {loss.item():.2f}')
            best_scalar = [scalar1.item(), scalar2.item()]
            best_offset = [offset1.item(), offset2.item()]
        # print(f'Loss: {loss.item():.2f}')
        if cnt >= max_step_count:
            break
    if cnt >= max_step_count:
        break

In [ ]:
import pandas as pd 
pd.Series(loss_arr).rolling(5).mean().plot(logy=True)

In [ ]:
best_scalar, best_offset

In [ ]:
class ScaledDataset:
    def __init__(self, dset, scalar, offset):
        self.dset = dset
        self.scalar = scalar
        self.offset = offset
    def __len__(self):
        return len(self.dset)
    def __getitem__(self, idx):
        inp, tar = self.dset[idx]
        return inp * self.scalar + self.offset, tar

In [ ]:
scaled_val_dset = ScaledDataset(val_dset, best_scalar[0], best_offset[0])

In [ ]:
pred_tiled, rec_loss, logvar_tiled, patch_psnr_tuple, pred_std_tiled = get_dset_predictions(model, scaled_val_dset,batch_size,
                                               num_workers=num_workers,
                                               mmse_count=mmse_count,
                                                model_type = config.model.model_type,
                                              )
tmp = np.round([x.item() for x in patch_psnr_tuple],2)
print('Patch wise PSNR, as computed during training', tmp,np.mean(tmp))

In [ ]:
pred = stitch_predictions(pred_tiled,val_dset)

In [ ]:
_,ax = plt.subplots(figsize=(10,5),ncols=2)
ax[0].imshow(pred[0][0,...,0])
ax[1].imshow(pred[0][0,...,1])

In [ ]:
from disentangle.scripts.evaluate import avg_range_inv_psnr
tar0 = [x[...,0] for x in tar_unnorm]
pred0 = [x[...,0] for x in pred]

tar1 = [x[...,1] for x in tar_unnorm]
pred1 = [x[...,1] for x in pred]
psnr_ch0 = avg_range_inv_psnr(np.concatenate(tar0,axis=0), np.concatenate(pred0, axis=0))
psnr_ch1 = avg_range_inv_psnr(np.concatenate(tar1,axis=0), np.concatenate(pred1, axis=0))

# stats_dict = compute_high_snr_stats(tar_unnorm, pred_unnorm)

In [ ]:
psnr_ch0, psnr_ch1